# Object Oriented Programming

Objects are _very_ powerful, and you can harness this power for your own designs!

## Writing a class

Writing a class should be second nature, and very natural to you - just as natural as writing a function!

In [ ]:
def f(x: float) -> float:
    return x ** 2


print(f"{f(3) = }")

In [ ]:
class F:
    def __call__(self, x: float) -> float:
        return x ** 2


f_inst = F()
print(f"{f_inst(3) = }")

## How classes work (Advanced, for self reading)

Classes are basically bags of objects with a few special features. So what separates them from a dict?

### The Type

All objects are "based" on a class (AKA type). You can call `type(object)` or `object.__class__` to get the class of an object. Types can "inherit" from other types, which just means they build on the other. The type is generally the "shared" part for objects, while the object itself holds the "unique" part that makes an instance special. Types tend to have methods, and instances tend to have member data, but since everything is an object, and functions really are just callable objects, the distinction between a variable and a function is a bit fuzzy. And that's okay.

Think of it this way: I am an instance of a Person. A Person inherits from a LivingThing. So I am an object, a Person is my class, and a Person inherits from LivingThing. You could have more levels of inheritance, like Mammal, if you want; in "real life", we tend to use this a lot, since it helps us structure our thinking. It helps structure our programs, too!

### The attribute names

There are several different ways to name attributes of your class or instance:

* `.some_item`: This is a publicly accessible attribute, users are expected to see it / use it. It could be a method, instance member, or a property - properties can be settable or read-only.
* `._some_item`: This is a "hidden" method - by social construct, users should not touch this, it's expected to be internal or "protected". Tab completion will usually ignore things that start with an `_`, unless you add the underscore then press tab. A subclass can access this.
* `.__some_item`: This is only (easily) accessible from _inside the current class_. Outside the class, including in a subclass, it's "mangled" by adding the class name. This is not intended to be "more secure" than a hidden method, but is intended for the special case where a class and a subclass need to define the same thing but have it different between them. Very rarely used.
* `.__some_item__`: This is a "dunder" attribute, and Python has many of them - they have special meaning to Python. It is not officially a good idea to add your own, as Python reserves them for future use, though some libraries (especially popular libraries) do add custom ones.
* Extra: `._some_item_`: If you need a "protocol", that is, a method that users should implement but should not directly call, then single underscores on both sides is an occasional convention that is not discouraged by Python authors, unlike adding new dunder methods. `_repr_html_` in IPython is an example of this convention in use.

### MRO (Method Resolution Order)

So, if you have an object `obj`, and you call `obj.something`, where does `something` come from, and is it a function or a variable? Hopefully, you know the answer to that last part of the question: there is no difference between a function and a variable; it's an object, and objects might be callable. The lookup order is called the MRO, or Method Resolution Order. It is:

1. **The object itself.** If the object has a `"something"` in `__slots__` or `__dict__`, then that's what you get.
2. **The class.** If it does not exist in the object, then it is pulled from the class.
3. **The parents of the class.** Then the parent (or parents) are checked. If there are multiple parents, the parents are checked left to right.
4. **The parent's parents (recursively).** Then the parent of parents are checked, all the way up until you hit `object`, which is the final parent and base for all classes.

You can see steps 2-4 using `.__mro__` on the class:

In [ ]:
F.__mro__

### Accessors

When you find a object, what do you actually return? Python has an accessors system. If the object has a `__get__` method (or `__set__` if you are setting something), then that is called with the instance as the argument and the result is returned. This is how methods work - (all) functions have a `__get__`, so when they are inside a class, they get called with `self` first; they return a "bound" method, one that will always include the instance in the call. This is how properties and staticmethod/classmethod work too, they customize `__get__`.

In [ ]:
F.__call__

In [ ]:
F.__call__(None, 2)

In [ ]:
f_inst.__call__

In [ ]:
f_inst.__call__(2)

We could to this by hand if we wanted to:

In [ ]:
F.__call__.__get__(f_inst)

### Special methods are accessed only on the class

Special methods are slightly special. If you perform an action that calls a special method, _it explicitly calls the class_, not the instance. So for example, `f(3)` calls `F.__call__(f, 3)`, _not_ `f.__call__(3)`. This was mostly an optimization, but it means you can't provide special behavior for an instance of a class that is not shared by the rest of the class. Which is probably good.

### Accessing something dynamic

Python lets you control this lookup with two special methods. One is `__getattr__`, which gets called if no attribute is found using the normal lookup method described above. You could use this to make a class that has dynamic methods beyond what it normally has. The second method is `__getattribute__`, which is called before anything else is looked up - you can literally do anything here, but at the cost that you have to then implement or revert to normal attribute lookup yourself to make the class usable, and simply getting at things like the data members in the class is a pain, since it will call itself. This should only be used in emergencies!

For setting, you don't need this split, so there's just `__setattr__`.

### `__dict__` vs. `__slots__`

Most user classes are `__dict__` classes, which store a dict on every instance. This is partially because it's what you get by default, and partially because it fits with the philosophy of Python.

In [ ]:
class Simple:
    a = 1

    def __init__(self):
        self.b = 2

In [ ]:
s = Simple()
s.__dict__

In [ ]:
s.c = 3
s.__dict__

In the above example, "b" is in the instance's `__dict__`. And so is `c`, once we add it. There's no limit to what we can add to the dictionary, really. Every instance of `Simple` could have a completely different `__dict__`. This is really bad design, of course - all instances should have the same data members (which means you should always assign everything initially in `__init__`.

By the way, what's `s.a`? It's a **class** variable. It's shared between all instances of the class, just like a method would be (remember, they are not different). You _very_ rarely need them; they are much more like global variables than anything else. The good news is that if you write `self.a = ...`, then `a` is added to the `__dict__`, and due to the look up I listed above, that's what gets used from now on for that instance, not the class variable. But, really, just don't do it.

Slots classes were mostly a performance/memory addition; storing the keys for every instance was a huge waste. In modern Python, there are optimizations under the hood, and `__dict__` classes do share keys when they can, but `__slots__` still has a nice correctness feature. Let's try the above with slots:

In [ ]:
class Strict:
    __slots__ = ("a", "b")


s = Strict()
s.a = 1
s.b = 2

In [ ]:
# s.c = 3
# s.__dict__

Now try to assign anything to the instance that is not in slots. It doesn't work! No more mistakes misspelling a member; it just won't work. And you can read the allowed members without digging into `__init__`! Notice there is no `__dict__` now.

The process for `__slots__` is rather weird, though. To avoid a `__dict__`, you need to have _every_ class in the MRO to have `__slots__`. If any parent doesn't have a slots, adding a slots would break it, so Python just ignores you if you inherit from a non-slots class. And to find a classes slots, you manually have to merge all `__slots__` in the MRO, it doesn't do it for you.

Conceptual question: Does `object` have `__slots__`?

> Super advanced aside: Just because I had not seen this mentioned before, and I _really_ needed it once: You can add `"__dict__"` (or `"__weakref__"`) to a `__slots__` list! This allows you to have a dictionary of items that can be added to for each instance _and_ have a fixed set of items that are not in that dictionary on each instance too. Just in case you ever need something like that, you now can't say you've never heard it before.

## Uses for classes

### Data + functions

They allow you to bundle data with the functions that run on them. In a language like Python without (much) type overloading, this is important for design (and good for tab completion).

In [ ]:
class Vector:
    def __init__(self, x: float, y: float) -> None:
        self.x = x
        self.y = y

    def mag(self) -> float:
        return (self.x ** 2 + self.y ** 2) ** 0.5

> Note: developing a class that represents data has a bit of boilerplate involved, so [attrs](https://www.attrs.org/en/stable/) (third-party) or [dataclasses](https://docs.python.org/3/library/dataclasses.html) (built-in for Python 3.7+, installable in 3.6) has a trick to make them _much more easily_, and automatically add things like nice repr's. We'll cover the parts of the syntax here later.

In [ ]:
# Built in for Python 3.7+, install otherwise
from dataclasses import dataclass


@dataclass
class Vector:
    x: float
    y: float

    def mag(self) -> float:
        return (self.x ** 2 + self.y ** 2) ** 0.5

In [ ]:
v = Vector(3, 4)
v.mag()

### Functors

I just told you it was a bad idea to set something outside your local scope, and often not even a good idea to just view something outside the local scope. So how do you write something that has scope? Use a class as a functor! Compare this:

In [ ]:
_start = 0


def incr() -> int:
    global _start
    _start += 1
    return _start

In [ ]:
incr()

In [ ]:
incr()

With this:

In [ ]:
class Incr:
    def __init__(self, incr: int = 0) -> None:
        self.incr = incr

    def __call__(self) -> int:
        self.incr += 1
        return self.incr

In [ ]:
incr = Incr()
incr()

In [ ]:
incr()

This is explicit, clear, I can have multiple instances without having them interfere, I can see exactly what's going on without having to trace down a global, and I can even set the default value when I make the `Incr` instance!

### Modularity

Let's say you have an algorithm with three parts. If you make a class that calls three member functions, you can allow a user to replace just one of the functions, and use the original first two. Classes are great for _organization_ and _code reuse_ because of this.

In [ ]:
class RunSomethingHard:
    def part1(self) -> None:
        print("Working hard")

    def part2(self) -> None:
        print("Working harder")

    def part3(self) -> None:
        print("That was hard!")

    def run(self) -> None:
        self.part1()
        self.part2()
        self.part3()

In [ ]:
inst = RunSomethingHard()
inst.run()

Now, look at how I can swap out part of the calculation without rewriting from scratch!

In [ ]:
class NewRunSomethingHard(RunSomethingHard):
    def part2(self) -> None:
        print("Nah, this is easy")

In [ ]:
inst = NewRunSomethingHard()
inst.run()

> Don't assume because something is on this list it is the _best_ way to solve a problem, it's just _a_ way, and something you'll likely commonly see (GEANT4 uses this heavily, for example). In this case, there may be other, better ways to share code depending on your problem. Think before you write!

### DSL (Domain Specific Language)

You can customize almost every behavior of a class to make them very natural for whatever you are doing.

In [ ]:
class Path(str):
    def __truediv__(self, other):
        return self.__class__(f"{self}/{other}")

In [ ]:
Path(".") / "myfile" / "program.py"

> Just in case you want to make a Path class like the one above - don't, use pathlib instead. We could have written `self.__class__` as `Path`, but then this would not subclass correctly and besides, using the class name inside the class is ugly and makes it harder to rename. If you return a normal string, then you can't keep applying `/`.
>
> Also, I left off type annotations for this example, as to do them properly I need to use a TypeVar.

### Mixins (advanced)

If you follow good practices, you can even make collections of behaviors and mix them into other classes - specifically, a mixin should not have an `__init__` or any new data members. (The second requirement is more important than the first, if you are careful to use `super()`. Let's rewrite the last example with mixins:

In [ ]:
class PathMixin:
    def __truediv__(self, other):
        return self.__class__(f"{self}/{other}")


class Path(str, PathMixin):
    pass


Path(".") / "myfile" / "program.py"

### Other: ABC, Protocols, and more

Other useful things to look into are ABCs (Abstract Base Classes) and Protocols. These let you a) require certain methods be implemented by users, and b) formalize "duck typing". ABC's are a run-time feature, and kind of half-broken; you have to instantiate an ABC class to get the benefit of the checking. Protocols are a "type-check time" feature, and are far better and don't require special inheritance, but only are enforced by type checkers (see a later section!).

## Design considerations

Object oriented programming has been known to make it easy to create spaghetti messes of code. The following tips will help you not fall into the trap and end up with poorly designed code. "Make it a class", by itself, will not magically make your code better.

### Modular design

You should break your code into _concepts_, and classes should help map those concepts to the computer. Different components of a detector might be classes, with an instance for each component. A vector, a URL, a remote data source, etc. You might have a class representing a unit of an analysis, and use either inheritance (okay) or a protocol (better, reduces coupling) to have real data processing vs. simulation generation. Etc.

### Unit test

We will mention testing later, and there is another course on it, but I'm focusing on the word _unit_. You need to be able to run your classes standalone, in unit tests, and not only in place. This keeps the design modular - you will resist the desire to make a class that needs a class to make another class inside a class that only works with the file that sits on your work laptop, etc. And you'll be free to redesign parts without having to worry about everything breaking down.

Always use PyTest for unit testing.

### Inheritance

You should be very cautious with Inheritance. It's a very powerful tool and should not be scary, but it's _so_ easy to misuse. These are the problems:

* It is often in direct contrast with modular design (you are linking things via inheritance) and hard to truly unit test (one class might not be testable without the other).
* It makes it harder to reason about where a method comes from when reading and debugging. Is it this class? A parent?

However, it is also _very_ useful. Read [this article](https://hynek.me/articles/python-subclassing-redux/) (which might be a little overly-harsh on subclassing, but it has good points) and ask yourself: Do you _need_ inheritance? Would composition or a Protocol work better? Certainly, many times, it is fine, but don't let the other considerations get swept aside if you use it.

### Functional design tips

**Classes with multiple states are bad**: If you have classes that init, then call some function like `.initialize()` or `.read_data()`, this is bad design. It's better to do the read in the `__init__` or as a `classmethod`. If you have two distinct states, then you have to design every method to be aware of both possible states! It's much better to either do the init at the beginning, or have two classes, one for each state.

**Never assign new members outside of __init__**: This is similar to the above point; if you assign a new member after init, then you now have two states; a "without" and a "with", so now you have to check `hasattr` in every other method, since you can't be sure what order the methods will be called in. Better to at least assign None in `__init__`. Or use `dataclasses`/`attrs`.

**Consider full functional design**: Python is object oriented, but it does support functional design, and it's worth thinking about it. Functional design has functions that are stateless (no mutable state), so you can chain them. So instead of:

```python
a = Thing()
a.init()
a.cleanup()
```

You would use:

```python
a = Thing().init().cleanup()
```

The only reason the first example could be better is due to memory usage/copying, which usually isn't a problem in Python. If it's an error to call `cleanup()` before `init()`, you can enforce that statically via classes in the second example, while the first example will just crash at runtime.

**Support static design wherever possible**: We will cover this later, but if you add things that are not statically sound, you probably should redesign. Simply supporting static typing tends to push you toward better design, and away from problems like those listed above.
